In [1]:
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

In [60]:
def load_data(temps=[-25,-15,-5,5,15,25,35,45]):
    
    data = dict()
    
    for temp in temps:
        
        if temp < 0:
            filename = '../data/P14_OCV/P14_OCV_N%d.mat' % (abs(temp))
        else:
            filename = '../data/P14_OCV/P14_OCV_P%d.mat' % (abs(temp))
        
        data[temp] = scipy.io.loadmat(filename, simplify_cells=True)['OCVData']
        
        print('Loaded: ',filename)
        
    return data

In [61]:
data = load_data()

Loaded:  ../data/P14_OCV/P14_OCV_N25.mat
Loaded:  ../data/P14_OCV/P14_OCV_N15.mat
Loaded:  ../data/P14_OCV/P14_OCV_N5.mat
Loaded:  ../data/P14_OCV/P14_OCV_P5.mat
Loaded:  ../data/P14_OCV/P14_OCV_P15.mat
Loaded:  ../data/P14_OCV/P14_OCV_P25.mat
Loaded:  ../data/P14_OCV/P14_OCV_P35.mat
Loaded:  ../data/P14_OCV/P14_OCV_P45.mat


In [67]:
def process_OCV25(data):
    
    scripts = ['script1','script2','script3','script4']
    total_discharge = 0
    total_charge = 0

    for script in scripts: 
        total_discharge += data[25][script]['disAh'][-1]
        total_charge += data[25][script]['chgAh'][-1]
        
    eta_25 = total_discharge / total_charge
    
    Q_25 = data[25]['script1']['disAh'][-1] \
     + data[25]['script2']['disAh'][-1] \
     - eta_25*data[25]['script1']['chgAh'][-1] \
     - eta_25*data[25]['script2']['chgAh'][-1]
    
    df_1 = pd.DataFrame(data[25]['script1'])
    
    # Beginning of discharge
    IR1Da = df_1[df_1['step'] == 1].tail(1).voltage.values[0] \
            - df_1[df_1['step'] == 2].head(1).voltage.values[0]
    
    # End of discharge
    IR2Da = df_1[df_1['step'] == 3].head(1).voltage.values[0] \
            - df_1[df_1['step'] == 2].tail(1).voltage.values[0]
    
    IndC = np.where(data[25]['script3']['step'] == 2)[0]
    
    # Beginning of charge
    IR1Ca = data[25]['script3']['voltage'][IndC[0]] \
        - data[25]['script3']['voltage'][IndC[0]-1]
    
    # End of charge
    IR2Ca = data[25]['script3']['voltage'][IndC[-1]] \
        - data[25]['script3']['voltage'][IndC[-1]+1]
    
    # These are just some sanity checks to keep these correction factors 
    # from getting too large in case of data issues 
    IR1D = min(IR1Da,2*IR2Ca)
    IR2D = min(IR2Da,2*IR1Ca)
    IR1C = min(IR1Ca,2*IR2Da)
    IR2C = min(IR2Ca,2*IR1Da)
    
    # Discharge voltage & SOC curves
    IndD = np.where(data[25]['script1']['step'] == 2)[0]
    blend = np.linspace(0,1,len(IndD))
    IRblend = IR1D + (IR2D-IR1D)*blend
    disV = data[25]['script1']['voltage'][IndD] + IRblend

    # Z = State of Charge
    disZ = 1 - data[25]['script1']['disAh'][IndD]/Q_25
    disZ = disZ + (1 - disZ[0])
    
    assert disZ.shape == disV.shape
    
    # Charge voltage & SOC curves
    blend = np.linspace(0,1,len(IndC))
    IRblend = IR1C + (IR2C-IR1C)*blend
    chgV = data[25]['script3']['voltage'][IndC] - IRblend
    chgZ = data[25]['script3']['chgAh'][IndC]/Q_25
    chgZ = chgZ - chgZ[0]
    
    assert chgZ.shape == chgV.shape
    
    charge_interpolate = interp1d(chgZ,chgV)
    discharge_interpolate = interp1d(disZ,disV)
    
    deltaV50 = charge_interpolate(0.5) - discharge_interpolate(0.5)
    
    # Select points on charge curve where SOC < 50%. 
    ind = np.where(chgZ < 0.5)[0]
    vChg = chgV[ind] - chgZ[ind]*deltaV50
    zChg = chgZ[ind]
    
    # Select points on charge curve where SOC > 50%.
    ind = np.where(disZ > 0.5)[0]
    vDis = (disV[ind] + ((1 - disZ[ind])*deltaV50))[::-1]
    zDis = (disZ[ind])[::-1]
    
    SOC = np.arange(0,1,0.005)
    rawocv_interpolation = interp1d(np.concatenate([zChg,zDis]), \
                            np.concatenate([vChg,vDis]), \
                            kind='linear',fill_value="extrapolate")
    rawocv = rawocv_interpolation(SOC)
    
    data[25]['rawocv'] = rawocv
    
    return data

In [68]:
def process_OCV(data,minV,maxV):
    

    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-68-a72b11140a10>, line 5)

In [69]:
test = process_OCV25(data)

In [70]:
test

{-25: {'script1': {'time': array([6.00071249e+01, 1.20021584e+02, 1.80036164e+02, ...,
          1.11262117e+05, 1.11263133e+05, 1.11264039e+05]),
   'step': array([1, 1, 1, ..., 3, 3, 3], dtype=uint8),
   'current': array([0., 0., 0., ..., 0., 0., 0.]),
   'voltage': array([4.14866066, 4.14849806, 4.14849806, ..., 2.88815331, 2.88994288,
          2.89140701]),
   'chgAh': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
   'disAh': array([ 0.        ,  0.        ,  0.        , ..., 14.44936529,
          14.44936529, 14.44936529])},
  'script2': {'time': array([   60.00328657,   120.01733791,   180.03140128, ...,
          21618.4102927 , 21619.42581441, 21620.316438  ]),
   'step': array([ 1,  1,  1, ..., 14, 14, 14], dtype=uint8),
   'current': array([0., 0., 0., ..., 0., 0., 0.]),
   'voltage': array([3.14357376, 3.14324832, 3.14276028, ..., 2.79997635, 2.79981375,
          2.79981375]),
   'chgAh': array([0.        , 0.        , 0.        , ..., 0.58241199, 0.58241199,
          0.5